In [1]:
import sys
sys.path.append('../')

In [2]:
from src.model import *
import copy
import torch

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
model = CompetenceModel.load('../models/all-mpnet-base-v2', 'biencoder', state_dict_path=None, device=device)

In [5]:
# Model prediction sample

sample_transcripts = ['First transcript', 'Second transcript']
sample_competence_sets = [
    ['Not competent', 'Half competent', 'Fully competent'], 
    ['Not competent', 'Half competent', 'Fully competent'],
]

with torch.no_grad():
    scores = model(sample_transcripts, sample_competence_sets)

print('Predict probabilities:')
print(scores)

Predict probabilities:
tensor([[0.2958, 0.3807, 0.3235],
        [0.2924, 0.4210, 0.2866]])


In [6]:
bayesian_model = BayesianCompetenceModel(model)

In [7]:
# PowerBALD acquisition on sample

sample_transcripts = ['First transcript', 'Second transcript', 
                      'Third transcript', 'Fourth transcript']
sample_competence_sets = [
    ['Not competent', 'Half competent', 'Fully competent'], 
    ['Not competent', 'Half competent', 'Fully competent'],
    ['Zero skill', 'One-third skill', 'Two-third skill', 'Skill'],
    ['Zero skill', 'One-third skill', 'Two-third skill', 'Skill'],
]

with torch.no_grad():
    scores = bayesian_model(sample_transcripts, sample_competence_sets, k=10)
    log_scores = torch.log(scores)
    batch = get_powerbald_batch(log_scores, batch_size=2)

print('Acquisition batch:', batch)

Entropy:   0%|          | 0/4 [00:00<?, ?it/s]

Conditional Entropy:   0%|          | 0/4 [00:00<?, ?it/s]

Acquisition batch: CandidateBatch(scores=[0.20952335496622862, 0.590555397101315], indices=[0, 1])
